In [7]:
import pandas as pd
import numpy as np
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

In [8]:
LOG_PATH = "detections_log.csv"  # ใช้ไฟล์ที่อัปโหลดมา
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720

In [3]:
def parse_xy(xy_str):
    try:
        xy = ast.literal_eval(str(xy_str))
        if isinstance(xy, (list, tuple)) and len(xy) == 2:
            cx = float(xy[0]) * FRAME_WIDTH
            cy = float(xy[1]) * FRAME_HEIGHT
            return cx, cy
    except:
        pass
    return None

In [4]:
# อ่าน CSV
df = pd.read_csv(LOG_PATH, on_bad_lines='skip')

# หาคอลัมน์ตำแหน่ง
xy_cols = [col for col in df.columns if "_xy" in col]

In [5]:
# หา frame id แรก
if "frame" in df.columns:
    first_frame_id = df["frame"].min()
    df_first_frame = df[df["frame"] == first_frame_id]
else:
    # ถ้าไม่มีคอลัมน์ frame ใช้แถวแรกของ log
    df_first_frame = df.iloc[[0]]

In [6]:
# รวมพิกัดเฉพาะเฟรมแรก
points = []
for _, row in df_first_frame.iterrows():
    for col in xy_cols:
        coords = parse_xy(row[col])
        if coords:
            points.append(coords)

points = np.array(points, dtype=float)
points = points[~np.isnan(points).any(axis=1)]

if len(points) < 2:
    raise ValueError(f"ต้องการอย่างน้อย 2 จุด แต่พบ {len(points)}")


ValueError: ต้องการอย่างน้อย 2 จุด แต่พบ 1

In [ ]:
# ทำ KDE plot
plt.figure(figsize=(10, 6))
sns.kdeplot(
    x=points[:, 0],
    y=points[:, 1],
    fill=True,
    cmap=cm.get_cmap("jet"),
    bw_adjust=0.5,
    levels=100,
    thresh=0,
    cbar=True
)
plt.gca().invert_yaxis()
plt.title("KDE Heatmap - First Frame Only")
plt.xlabel("X position (px)")
plt.ylabel("Y position (px)")
plt.savefig("heatmap_kde_first_frame.png", dpi=300)
plt.show()

print("บันทึก heatmap_kde_first_frame.png แล้ว")